# Understand Number Spaces - Similar Mode

Our `NumberSpace` has a SIMILAR mode too, enabling search for similar numbers. 

Check MinMax mode [here](number_embedding_minmax.ipynb).

In [1]:
%pip install superlinked==3.8.0

In [2]:
import pandas as pd

from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import Integer
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.number_space import NumberSpace, Mode

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

## Similar mode

In this case we prefer numbers that are similar to the number we are going to query with.

In [3]:
@schema
class Person:
    id: IdField
    height: Integer  # set height in cms for example


person = Person()

height_space = NumberSpace(
    number=person.height, min_value=100, max_value=220, mode=Mode.SIMILAR
)  # height out of this range is considered an outlier in our usecase
person_index = Index(height_space)

In [4]:
source: InMemorySource = InMemorySource(person)
executor = InMemoryExecutor(sources=[source], indices=[person_index])
app = executor.run()

In [5]:
source.put([{"id": "person1", "height": 175}, {"id": "person2", "height": 180}])

In [6]:
query = Query(person_index).find(person).similar(height_space.number, 178)

In [7]:
result = app.query(query)

pd.DataFrame([entry.stored_object for entry in result.entries])

,height,id
0,180,person2
1,175,person1


As expected, the person 180 cm high comes first (as their height is closer to the queried number, 178), and the person with height of 175 cm comes second.